In [1]:
import utilitarios as utils # Funciones propias
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import math

### Semilla

In [2]:
random_seed = 2022 # Number of life :)

### K-fold Cross Validation

In [26]:
def test_k_fold(min_cuts , max_cuts, max_folds, neighbors, k_fold_func):
    X, y, X_raw = utils.get_data(src_dir="Data_preprocesada/", iterations= min_cuts-1)

    dataframe = []
    columns = ['Number of cuts', 'Length of X', 'K fold', 'Neighbors',
                'Train error', 'Train var. error', 'Train bias error', 'Train list errors',
                'Test error', 'Test var. error', 'Test bias error', 'Test list errors',
                ]

    for iteration in range(min_cuts, max_cuts+1):
        X, X_raw = utils.iterate_data(X_raw)
        X = utils.normalization(X)
        X_length = len(X[0])


        for number_of_folds in range(2, max_folds+1):
            k_folds_data = k_fold_func(X, y, number_of_folds, random_seed)

            for neighbor in neighbors:
                data_aux = [iteration, X_length, number_of_folds, neighbor]

                clf = KNeighborsClassifier(n_neighbors=neighbor)

                error_train = 0
                bias_train = 0
                varianza_train = 0
                list_of_errors_train = []

                error_test = 0
                bias_test = 0
                varianza_test = 0
                list_of_errors_test = []

                for i in range(number_of_folds):
                    clf.fit(k_folds_data[i]['X_train'], k_folds_data[i]['y_train'])

                    error_i_train = 1 - clf.score(k_folds_data[i]['X_train'], k_folds_data[i]['y_train'])
                    error_i_test = 1 - clf.score(k_folds_data[i]['X_test'], k_folds_data[i]['y_test'])

                    error_i_train = round(error_i_train,6)
                    error_i_test = round(error_i_test,6)

                    list_of_errors_train.append(error_i_train)
                    list_of_errors_test.append(error_i_test)

                error_train = sum(list_of_errors_train) / len(list_of_errors_train)
                error_test = sum(list_of_errors_test) / len(list_of_errors_test)


                varianza_train = sum((x-error_train)**2 for x in list_of_errors_train) / len(list_of_errors_train)
                varianza_test = sum((x-error_test)**2 for x in list_of_errors_test) / len(list_of_errors_test)

                bias_train = math.sqrt(error_train - varianza_train)
                bias_test = math.sqrt(error_test - varianza_test)


                data_aux.extend([error_train,varianza_train, bias_train,list_of_errors_train])
                data_aux.extend([error_test,varianza_test, bias_test,list_of_errors_test])

                dataframe.append(data_aux)

    dataframe_estratificado = pd.DataFrame(data = dataframe, columns = columns)
    return dataframe_estratificado

In [30]:
df = test_k_fold(min_cuts=7, max_cuts=7, max_folds=3, neighbors=[1, 3], k_fold_func=utils.get_stratified_k_fold_cross_validation)
df

,Number of cuts,Length of X,K fold,Neighbors,Train error,Train var. error,Train bias error,Train list errors,Test error,Test var. error,Test bias error,Test list errors
0,7,60,2,1,0.000000,0.000000,0.000000,"[0.0, 0.0]",0.602020,7.211910e-06,0.775895,"[0.599335, 0.604706]"
1,7,60,2,3,0.376610,0.000074,0.613625,"[0.385193, 0.368026]",0.681776,4.329640e-07,0.825697,"[0.681118, 0.682434]"
2,7,60,3,1,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0]",0.562974,5.396511e-06,0.750312,"[0.559948, 0.563378, 0.565595]"
3,7,60,3,3,0.347232,0.000023,0.589244,"[0.353751, 0.345589, 0.342356]",0.649197,1.560280e-05,0.805718,"[0.643636, 0.652439, 0.651515]"
